In [ ]:
!pip install --user -U nltk

import nltk
import numpy as np
import pandas as pd
import time
import warnings


from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.under_sampling import RandomUnderSampler

from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import BernoulliNB, GaussianNB, ComplementNB, MultinomialNB
from sklearn.preprocessing import scale, StandardScaler, PolynomialFeatures, Normalizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression

warnings.filterwarnings("ignore")
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rst20\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Датасет был заранее скачан с кегла на гуглодиск

In [ ]:
df_train = pd.read_csv('https://drive.google.com/uc?id=1_tZHuLfFz5_xUau-Pr-1grmzSkEYnleb', sep=',')
df_test = pd.read_csv('https://drive.google.com/uc?id=11Z11g5fwGgAsrasmV-AC5Yi0_DGdetA7', sep=',')
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


Так как даная задача связана с анализом текстов (что и подтверждает датасет), то обычные шаги по предварительному анализу и очистке датасета представляются излишними: векторизация представит данные в виде сотен тысяч признаков, которые ничего не скажут человеческому глазу, а пытаться выделить выбросы автоматическими методами, основанными на поиске ближайших соседей --- крайне затратное с точки зрения времени работы мероприятие.

В описании датасета утверждается, что он размечен на две категории

In [ ]:
print(df_train.info())
print(df_train['label'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB
None
label
0    29720
1     2242
Name: count, dtype: int64


Распределение классов несбалансировано: сильно смещено в сторону класса 0.

In [ ]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


Данные во втором файле, названном авторами датасета тестовыми, не содержат колонку label, поэтому мы их с негодованием отвергаем, и в дальнейшем речи о них здесь не будет.

Для того, чтобы хоть как-то уменьшить объём работы алгоритмам, загрузим стоп-слова на английском: в дельнейшем будем передавать их векторизаторам.

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

Классический подход: Bag of words

In [ ]:
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3)).fit(df_train['tweet'])

X_all_data = vectorizer.transform(df_train['tweet'])
Y_all_data = df_train['label']
print('размер словаря:', len(vectorizer.vocabulary_), 'размерность данных:', X_all_data.shape)
elem = X_all_data[0]
print('векторизованное значение:', elem[elem.nonzero()])

# работает очень долго
#projector = NMF(n_components=100)
#X_truncated = projector.fit_transform(X_all_data)
#print('размерность данных:', X_truncated.shape)
#print(X_truncated[0])

X_train, X_test, y_train, y_test = train_test_split(X_all_data, Y_all_data, test_size=0.2, shuffle=True)

размер словаря: 374556 размерность данных: (31962, 374556)
векторизованное значение: [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


Простейшая проверка работы алгоритмов:

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.90      0.94      5994
           1       0.34      0.77      0.47       399

    accuracy                           0.89      6393
   macro avg       0.66      0.84      0.71      6393
weighted avg       0.94      0.89      0.91      6393



В предыдущих заданиях масштабирование данных помогало улучшить точность предсказаний. Попробуем аналогичный подход для наших данных и наших алгоритмов:

In [ ]:
X_norm = Normalizer().fit_transform(X_all_data)
elem = X_norm[0]
print('+ масштабирование:', elem[elem.nonzero()])

X_train, X_test, y_train, y_test = train_test_split(X_norm, Y_all_data, test_size=0.2, shuffle=True)
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

+ масштабирование: [[0.24314167 0.16896196 0.04678636 0.24314167 0.24314167 0.20052484
  0.15407348 0.24314167 0.24314167 0.13898553 0.24314167 0.24314167
  0.11550985 0.24314167 0.24314167 0.24314167 0.24314167 0.23391012
  0.24314167 0.24314167 0.22227974]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5947
           1       1.00      0.10      0.18       446

    accuracy                           0.94      6393
   macro avg       0.97      0.55      0.57      6393
weighted avg       0.94      0.94      0.91      6393



Неудачный опыт.

Попробуем сбалансировать данные.
Количество признаков для наших данных после векторизации довольно велико, поэтому оверсемплинг представляется не самой удачной идеей.
А вот уменьшить количество экземпляров класса большей мощности можно достаточно легко (в вычислительном смысле)

In [ ]:
print('исходное распределение по классам:', np.unique(y_train, return_counts=True))
sampler = RandomUnderSampler(sampling_strategy=0.25)
# параметр sampling_strategy у андерсэмплера достоин подбора
X_train_balanced, y_train_balanced = sampler.fit_resample(X_train, y_train)
print('скорректированное распределение по классам:', np.unique(y_train_balanced, return_counts=True))

исходное распределение по классам: (array([0, 1], dtype=int64), array([23773,  1796], dtype=int64))
скорректированное распределение по классам: (array([0, 1], dtype=int64), array([7184, 1796], dtype=int64))


Сравним различные варианты алгоритмов на основе "наивного" применения теоремы Байеса:

In [ ]:
def check_classifier(clf):
    print(type(clf).__name__)
    clf.fit(X_train_balanced, y_train_balanced)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))

check_classifier(MultinomialNB())
check_classifier(ComplementNB())
check_classifier(BernoulliNB())
print('параметры по умолчанию дают несколько странные результаты')
check_classifier(BernoulliNB(alpha=0.1))
# параметр alpha у классификатора достоин подбора

MultinomialNB
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5947
           1       1.00      0.18      0.30       446

    accuracy                           0.94      6393
   macro avg       0.97      0.59      0.64      6393
weighted avg       0.95      0.94      0.92      6393

ComplementNB
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      5947
           1       0.62      0.70      0.66       446

    accuracy                           0.95      6393
   macro avg       0.80      0.83      0.82      6393
weighted avg       0.95      0.95      0.95      6393

BernoulliNB
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5947
           1       0.00      0.00      0.00       446

    accuracy                           0.93      6393
   macro avg       0.47      0.50      0.48      6393
weighted avg       0.87      0.93  

С учётом экспериментов выше, попробуем подобрать параметры:

In [ ]:
model = Pipeline(steps=[
    ('undersampler', RandomUnderSampler()),
    ('classifier', AdaBoostClassifier(n_estimators=20))
])
params_grid = {
    'undersampler__sampling_strategy': np.array(np.linspace(0.1, 1, 10), dtype='float32'),
    'classifier__estimator': [MultinomialNB(force_alpha=True), ComplementNB(force_alpha=True), BernoulliNB(force_alpha=True)],
    'classifier__estimator__alpha':  np.array(np.linspace(0, 0.1, 10), dtype='float32'),
}
start = time.perf_counter()
gs = GridSearchCV(model, params_grid, cv=5, n_jobs=-1, verbose=3)
gs.fit(X_train, y_train)
print('подбор параметров:', time.perf_counter() - start, 'с')

print(gs.best_params_, gs.best_estimator_)
print(classification_report_imbalanced(y_test, gs.predict(X_test)))


Fitting 5 folds for each of 300 candidates, totalling 1500 fits
подбор параметров: 251.31895650000024 с
{'classifier__estimator': MultinomialNB(force_alpha=True), 'classifier__estimator__alpha': 0.011111111, 'undersampler__sampling_strategy': 0.4} Pipeline(steps=[('undersampler', RandomUnderSampler(sampling_strategy=0.4)),
                ('classifier',
                 AdaBoostClassifier(estimator=MultinomialNB(alpha=0.011111111,
                                                            force_alpha=True),
                                    n_estimators=20))])
                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.99      0.41      0.97      0.63      0.43      5924
          1       0.74      0.41      0.99      0.52      0.63      0.38       469

avg / total       0.94      0.95      0.45      0.94      0.63      0.42      6393



Попытка улучшить алгоритм с помощью адаптивного бустинга дала не особо хорошие результаты. Параметр f1 для лучшего из вариантов хуже, чем лучший из опробованных выше алгоритмов с параметрами по умолчанию.
Попробуем ещё раз подобрать парамтеры, но теперь упакуем классификаторы в BaggingClassifier:

In [ ]:
model = Pipeline(steps=[
    ('undersampler', RandomUnderSampler()),
    ('classifier', BaggingClassifier())
])
params_grid = {
    'undersampler__sampling_strategy': np.array(np.linspace(0.1, 1, 10), dtype='float32'),
    'classifier__estimator': [MultinomialNB(force_alpha=True), ComplementNB(force_alpha=True), BernoulliNB(force_alpha=True)],
    'classifier__estimator__alpha':  np.array(np.linspace(0, 0.1, 10), dtype='float32'),
}
start = time.perf_counter()
gs = GridSearchCV(model, params_grid, cv=5, n_jobs=-1, verbose=3)
gs.fit(X_train, y_train)
print('подбор параметров:', time.perf_counter() - start, 'с')

print(gs.best_params_, gs.best_estimator_)
print(classification_report_imbalanced(y_test, gs.predict(X_test)))


Fitting 5 folds for each of 300 candidates, totalling 1500 fits
подбор параметров: 126.2759087000004 с
{'classifier__estimator': MultinomialNB(force_alpha=True), 'classifier__estimator__alpha': 0.1, 'undersampler__sampling_strategy': 0.3} Pipeline(steps=[('undersampler', RandomUnderSampler(sampling_strategy=0.3)),
                ('classifier',
                 BaggingClassifier(estimator=MultinomialNB(alpha=0.1,
                                                           force_alpha=True)))])
                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.99      0.63      0.98      0.79      0.65      5924
          1       0.81      0.63      0.99      0.71      0.79      0.60       469

avg / total       0.96      0.96      0.66      0.96      0.79      0.64      6393



Всё стало заметно лучше. И к тому же, за меньшее время.

Попробуем обучить алгоритм "с помощью классических от TruncatedSVD на основе полиномиальных от tf-idf с (1-3) биграммами".

Так как TruncatedSVD возвращает признаки, которые не являются неотрицательными, то с этим инструментом понижения размерности не получится использовать алгоритмы на основе наивного Байеса: но зато есть множество других алгоритмов.

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 3)).fit(df_train['tweet'])
X_all_data = vectorizer.transform(df_train['tweet'])
Y_all_data = df_train['label']

projector = TruncatedSVD(n_components=100, algorithm='randomized')
start = time.perf_counter()
X_truncated = projector.fit_transform(X_all_data)
print('уменьшение размерности:', time.perf_counter() - start, 'с')

scaler = StandardScaler()
start = time.perf_counter()
X_scaled = scaler.fit_transform(X_truncated)
print('масштабирование:', time.perf_counter() - start, 'с')

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y_all_data, test_size=0.2, shuffle=True)

sampler = RandomUnderSampler(sampling_strategy=0.25)
start = time.perf_counter()
X_train_balanced, y_train_balanced = sampler.fit_resample(X_train, y_train)
print('андерсемплинг:', time.perf_counter() - start, 'с')

def check_classifier(clf):
    print(type(clf).__name__)
    b_clf =BaggingClassifier(estimator=clf)
    start = time.perf_counter()
    b_clf.fit(X_train_balanced, y_train_balanced)
    print('обучение классификатора:', time.perf_counter() - start, 'с')
    y_pred = b_clf.predict(X_test)
    print(classification_report(y_test, y_pred))

check_classifier(LinearSVC(dual=False))
check_classifier(SGDClassifier())
check_classifier(LogisticRegression())
check_classifier(RandomForestClassifier())
check_classifier(RandomForestClassifier(n_estimators=10))


уменьшение размерности: 9.355633600000147 с
масштабирование: 0.04042459999982384 с
андерсемплинг: 0.007768800000121701 с
LinearSVC
обучение классификатора: 3.949679199999082 с
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5962
           1       0.53      0.34      0.41       431

    accuracy                           0.94      6393
   macro avg       0.74      0.66      0.69      6393
weighted avg       0.92      0.94      0.93      6393

SGDClassifier
обучение классификатора: 0.5941450000009354 с
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      5962
           1       0.46      0.39      0.43       431

    accuracy                           0.93      6393
   macro avg       0.71      0.68      0.69      6393
weighted avg       0.92      0.93      0.93      6393

LogisticRegression
обучение классификатора: 0.7008351999993465 с
              precision    recall  f1-score  

Прямо скажем, точность результатов не впечатляет. Возможно, подбором параметров удастся повысить качество работы алгоритма, но чтобы превзойти результаты, полученные наивным байесом придётся перебирать очень большое пространство параметров без какой-либо гарантии успеха.

Итого: лучшие результаты удалось получить следующим алгоритмом:

{

'classifier__estimator': MultinomialNB(force_alpha=True),

'classifier__estimator__alpha': 0.1,

'undersampler__sampling_strategy': 0.3

}

Pipeline(steps=\[
	('undersampler', RandomUnderSampler(sampling_strategy=0.3)),
	('classifier', BaggingClassifier(estimator=MultinomialNB(alpha=0.1, force_alpha=True)))
	\])